In [ ]:
import cv2
import os, glob
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
from dataset import *
import pandas as pd
from PIL import Image
from torchvision import transforms
from torchvision import models as tm
from efficientnet_pytorch import EfficientNet
from model import *
from datetime import datetime
from torch.nn.parallel import DataParallel
import tqdm
import torchattacks
import random as rn
##=============Config===================
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
device='cuda'
now = datetime.now()
# dd/mm/YY
val_batch_size=256
root = "/ssd3/ilsvrc2012/"
val_fn = root+'val.txt'



### original inference process ###

stmodel= tm.resnet101(pretrained=True).to(device)
imTrs = imageTrans(device).to(device)
stmodel = DataParallel(stmodel)
imTrs = DataParallel(imTrs)

stmodel.load_state_dict(torch.load('checkpoint/final_net.pt'))
imTrs.load_state_dict(torch.load('checkpoint/final_imt.pt'))

val_dataset = dataset_DFD(0, val_fn, root=root+'val/', mode='val')
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=val_batch_size, drop_last=True, 
                                         num_workers=16, sampler=None, shuffle=False)


atk = torchattacks.PGD(nn.Sequential(imTrs,stmodel), eps=4/255, alpha=1/255, steps=200)
target_map_function = lambda images, labels: labels.fill_(300)
atk.set_attack_mode("targeted", target_map_function=target_map_function)

# atk.save(data_loader=val_loader, save_path="./imagenet_val_pgd.pt", verbose=True)


stmodel.eval()
imTrs.train()
val_acc = 0
print('Start validating')
opt_inner = torch.optim.AdamW(imTrs.parameters(), lr=3e-5)
mse=torch.nn.MSELoss().type(torch.cuda.FloatTensor)

val_acc = {}
denoise_step=40
for i in range(denoise_step):
    val_acc[i] = []

total_step =len(val_loader)
for outer_step, (X,y) in enumerate(val_loader):
    X = X.to(device)
    y = y.to(device)
    X = atk(X, y)

    ## Inner optimization====
    
    for inner_ep in range(denoise_step):
        opt_inner.zero_grad()
        X_in = imTrs(X, snr=15).to(device)
        loss_inner = mse(X_in, X)
        loss_inner.backward()
        opt_inner.step()
        
        with torch.no_grad():
            pred = stmodel(imTrs(X_in).detach())
            _, pred = torch.max(pred, 1)
            val_acc[inner_ep].append( (pred == y).sum().item()/y.size(0))

    ##======================
    if outer_step % 4==0:
        print('.', end='')
print()
        
now = datetime.now()
d1 =now.strftime("%d/%m/%Y %H:%M:%S")
for key in list(val_acc.keys()):
    print("[%s] Validation Accuracy=%.5f" % (d1, np.mean(np.array(val_acc[key]))))


In [ ]:

[02/01/2021 22:59:13] Validation Accuracy=0.52548
snr30